# Лабораторная работа №3
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def calc_prefix_func(s: str) -> list[int]:
    n = len(s)
    res = [0] * n
    for i in range(1, n):
        j = res[i - 1]
        while (j > 0 and s[i] != s[j]):
            j = res[j - 1]
        if (s[i] == s[j]):
            j += 1
        res[i] = j
    return res

print(calc_prefix_func("abcaaabc"))

[0, 0, 0, 1, 1, 1, 2, 3]


In [3]:
def find_substring_1(t: str, s: str) -> int:
    n = len(t)
    p = calc_prefix_func(t + '#' + s)
    for i in range(n + 1, len(p)):
        if p[i] == n:
            return i - 2*n
    return -1

#### Упрощенный алгоритм Бойера-Мура

In [4]:
from typing import DefaultDict
def get_bias_table(s: str) -> DefaultDict[str, int]:
    n = len(s)
    d = defaultdict(lambda: n)
    d[s[-1]] = n
    for i in range(-2, -n - 1, -1):
        if s[i] not in d:
            d[s[i]] = -i - 1
    return d

def find_substring_2(t: str, s: str) -> int:
    bias_table = get_bias_table(t)
    i = len(t) - 1
    while i < len(s):
        for j in range(len(t)):
            if s[i-j] != t[-j-1]:
                i += bias_table[s[i]]
                break
            return i - len(t) + 1
    return -1

print(find_substring_2("abbc", "adxbgsfgbabbcd"))
            

9


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 


In [2]:
test_x = [12,8 ,4, 13, 14, 11, 7, 9, 15, 10, 6, 1, 5, 2, 3, 0] 

In [3]:
from abc import ABC, abstractmethod

class SearchClass(ABC):
    def __init__(self, x): # x должен быть осортированным списком
        self._x = x
        super().__init__()
            
    @abstractmethod
    def _find_insertion_index(self, el, key):
        pass
        
    def find(self, el, key):
        i = self._find_insertion_index(el, key)
        assert key(el) == key(self._x[i]), "Element not found"
        return i
    
    def insert(self, el, key):
        index = self._find_insertion_index(el, key)
        self._x.insert(index, el)
        return index
    
    def remove(self, el, key):
        i = self.find(el, key)
        self._x.pop(i)
        return i
    
    def __str__(self) -> str:
        return str(self._x)

class InterpolationSearch(SearchClass):
    def _find_insertion_index(self, el, key):
        el_key = key(el)
        x = self._x
        n = len(x)
        if n == 0:
            return 0
        l, r = 0, n - 1
        while key(x[l]) < el_key < key(x[r]):
            m = l + (el_key-key(x[l])) * (r-l) // (key(x[r])-key(x[l]))
            if key(x[m]) < el_key:
                l = m + 1
            elif key(x[m]) > el_key:
                r = m - 1
            else:
                return m
        if key(x[l]) == el_key:
            return l
        elif key(x[r]) == el_key: 
            return r
        else:
            return r + 1 if key(x[r]) - el_key < el_key - key(x[l]) else l 

In [4]:
from __future__ import annotations
from functools import cache

def swap(field: list[int], a_i: int, b_i: int):
    res = field.copy()
    res[a_i], res[b_i] = res[b_i], res[a_i]
    return res

def get_dist(a_i: int, b_i: int) -> int:
    return abs(a_i // 4 - b_i // 4) + abs(a_i % 4 - b_i % 4)



class Position:
    def __init__(self, field: list[int], z_i: int, path: list[int]):
        self.field = field
        self.path = path
        self.z_i = z_i
        self.dist = self._calc_all_dists() 
        self.func = self.dist + len(path)
    
    def get_positions(self) -> list[Position]:
        res: list[Position] = []
        z_i = self.z_i
        if z_i % 4 > 0:
            res.append(Position(swap(self.field, z_i, z_i - 1),
                             z_i - 1,
                             self.path + [self.field[z_i - 1]]))
        if self.z_i % 4 < 3:
            res.append(Position(swap(self.field, z_i, z_i + 1),
                             z_i + 1,
                             self.path + [self.field[z_i + 1]]))
        if self.z_i // 4 < 3:
            res.append(Position(swap(self.field, z_i, z_i + 4),
                             z_i + 4,
                             self.path + [self.field[z_i + 4]]))
        if self.z_i // 4 > 0:
            res.append(Position(swap(self.field, z_i, z_i - 4),
                             z_i - 4,
                             self.path + [self.field[z_i - 4]]))
        return res
    
    def _calc_all_dists(self) -> int:
        res = 0
        for i, n in enumerate(self.field):
            if n == 0:
                continue
            res += get_dist(i, n - 1)
            if (y := i // 4) == (y_g := (n - 1) // 4):
                if y < y_g:
                    for j in range(y + 1, y_g + 1):
                        if self.field[j] < y:
                            res += 2
                            break
                else:
                    for j in range(y_g, y):
                        if self.field[j] > y:
                            res += 2
                            break
            if (x := i % 4) == (x_g := (n - 1) % 4):
                if x < x_g:
                    for j in range(x + 1, x_g + 1):
                        if self.field[j] < x:
                            res += 2
                            break
                else:
                    for j in range(x_g, x):
                        if self.field[j] > x:
                            res += 2
                            break
            
        return res
    def __hash__(self) -> int:
        return hash(tuple(self.field))
    
    def __str__(self) -> str:
        str_field = list(map(str, self.field))
        return '\n'.join(' '.join(str_field[i:i+4]) for i in range(0, 15, 4))

    


In [7]:
def task2(x: list[int]) -> list[int]:
    t = x[::]
    t = list(filter(bool, t[:4] + t[4:8][::-1] + t[8:12] + t[12:17][::-1]))
    k = len([(i,j) for i in range(14) for j in range(i, 15) if t[i] > t[j]])
    res = []
    if k % 2 == 0:
        return res
    z_i = x.index(0)
    opened_positions_list: list[Position] = [Position(x, z_i, [])]
    interp_search = InterpolationSearch(opened_positions_list)
    opened_positions: set[Position] = {opened_positions_list[0]}
    closed_positions: set[Position] = set() 
    min_dist = 9999
    while opened_positions_list[0].dist > 0:
        if opened_positions_list[0].dist < min_dist:
            min_dist = opened_positions_list[0].dist
            print(min_dist)
            print(opened_positions_list[0])
        new_positions = opened_positions_list[0].get_positions()
        opened_positions.remove(opened_positions_list[0])
        closed_positions.add(opened_positions_list.pop(0)) 
        for pos in new_positions:
            if pos in closed_positions or pos in opened_positions:
                continue
            interp_search.insert(pos, key=lambda x: x.func)
            opened_positions.add(pos)
    print(opened_positions_list[0].path)
    return opened_positions[0].path
    
    
            
        
    


In [8]:
task2(test_x)

42
12 8 4 13
14 11 7 9
15 10 6 1
5 2 3 0
41
12 8 4 0
14 11 7 13
15 10 6 9
5 2 3 1
40
12 8 0 4
14 11 7 13
15 10 6 9
5 2 3 1
39
12 0 8 4
14 11 7 13
15 10 6 9
5 2 3 1
38
0 12 8 4
14 11 7 13
15 10 6 9
5 2 3 1
37
12 0 8 4
14 6 7 13
15 11 10 9
5 2 3 1
36
0 12 8 4
14 6 7 13
15 11 10 9
5 2 3 1
35
12 0 8 4
14 6 7 13
15 1 11 9
5 10 2 3
34
0 12 8 4
14 6 7 13
15 1 11 9
5 10 2 3
33
1 0 8 4
12 14 7 13
15 11 10 9
5 2 6 3
32
12 2 8 4
11 0 7 13
5 10 6 9
14 15 3 1
31
12 2 8 4
0 11 7 13
5 10 6 9
14 15 3 1
30
0 2 8 4
12 11 7 13
5 10 6 9
14 15 3 1
29
1 12 8 4
0 6 7 13
14 15 11 9
5 10 2 3
28
0 2 8 4
12 10 7 13
5 6 11 9
14 15 3 1
27
12 2 8 4
5 11 7 13
10 6 3 9
0 14 15 1
26
0 12 7 4
9 6 8 1
14 10 13 11
5 2 15 3
25
1 12 8 4
0 6 7 13
14 10 11 9
5 2 15 3


KeyboardInterrupt: 

### Вывод